# MLP Coding Example

Acesse o servidor remoto por ssh. Crie uma virtualenv com:
```
mkvirtualenv <nome-da-sua-env>
```
Ative a sua virtualenv com:
```
workon <nome-da-sua-env>
```
Instale o jupyter:
```
pip install jupyter
```
Na pasta contendo o setup.py, instale o pacote do projeto :
```
pip install -e .
```
Comando para servir o jupyter:
```
nohup jupyter notebook --no-browser &
```
Talvez você precise de um token. Se precisar consulte com:
```
jupyter notebook list
```




Na sua máquina local, redirecione a porta adequada:
```
ssh -NfL localhost:<porta-local>:localhost:<porta-remoto> <seu-usuario>@<ip-do-servidor>
```
Geralmente:
```
ssh -NfL localhost:8888:localhost:8888 <seu-usuario>@<ip-do-servidor>
```
Abra localhost:8888 no seu browser. Se você quiser fechar o jupyter, no localhost:8888 clique em Quit, depois libere a porta com:
```
lsof -ti:8888 | xargs kill -9
```

## Imports

In [1]:
import numpy as np
from numpy import genfromtxt
from tqdm.notebook import tqdm
from perceptronac.context_training import context_training
from perceptronac.context_coding import context_coding
from perceptronac.perfect_AC import perfect_AC
import torch
import os

## Gerando dados randômicos correlacionados (substituir pelos seus dados)

In [37]:
#X is context and y is bitstream to encode
# parameters  
L = 100000 # how many samples 
N = 7 # order of the AR
# Np = N # number of parameters to estimate 

C0 = np.random.rand(1,1) 
C = np.random.rand(N,1)

X = 2 * (np.random.rand(2*L,N) > 0.5) - 1 # correlated (context) signals

X = (X > 0).astype(int)

def sigmoid(x): 
    return 1 / (1 + np.e**(-x))

p = sigmoid(C0 + X @ C);
yy = (np.random.rand(2*L, 1) > (1 - p)).astype(int) # signal 
yt = yy[0:L] > 0 # train on the first part 
yc = yy[L:L+L] > 0 # encode the second part
Xt = X[0:L,0:N] # truncated X for training 
Xc = X[L:L+L,0:N] # truncated X for coding

In [52]:
#Xc

array([[ 0, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1],
       [ 1, -1, -1, -1, -1],
       ...,
       [-1, -1,  1, -1, -1],
       [ 1, -1, -1, -1, -1],
       [-1, -1,  0, -1, -1]])

In [53]:
#X is context and y is bitstream to encode
L = 100000 # how many samples 
N = 5 # order of the AR
my_data = genfromtxt('../SPIHT_dataset/SPIHT_bits_with_context_ricardo14.csv', delimiter=',')
bitstream = my_data[:, 0].astype(int)
context = my_data[:, 1:6].astype(int)
bitstream = bitstream.reshape((len(bitstream), 1))



yt = bitstream[0:L] > 0 # train on the first part 
yc = bitstream[L:L+L] > 0 # encode the second part
Xt = context[0:L,0:N] # truncated X for training 
Xc = context[L:L+L,0:N] # truncated X for coding

In [54]:
#context

## Entropia dos dados

In [55]:
# treino
perfect_AC(yt,context_coding(Xt,context_training(Xt,yt)))

3.203426503814918e-16

In [56]:
# teste
perfect_AC(yc,context_coding(Xc,context_training(Xc,yc)))

3.203426503814918e-16

## Treinando Modelo No Pytorch com Batch Gradient Descent (Quando todos os dados couberem na memória da placa de vídeo de uma só vez)

In [57]:
import torch

In [58]:
class Perceptron(torch.nn.Module):
    def __init__(self,N):
        super().__init__()
        self.linear = torch.nn.Linear(N, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

In [59]:
class Log2BCELoss(torch.nn.Module):
    def __init__(self,*args,**kwargs):
        super().__init__()
        self.bce_loss = torch.nn.BCELoss(*args,**kwargs)

    def forward(self, pred, target):
        return self.bce_loss(pred, target)/torch.log(torch.tensor(2,dtype=target.dtype,device=target.device))

In [60]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self,X,y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.y)
    def __getitem__(self,idx):
        return self.X[idx,:],self.y[idx,:]

In [61]:
net = Perceptron(N)

In [62]:
net = net.cuda()

In [63]:
trainset = CustomDataset(Xt,yt)
validset = CustomDataset(Xc,yc)

In [64]:
criterion = Log2BCELoss(reduction="sum")
optimizer = torch.optim.SGD(net.parameters(), lr=0.00001)

In [65]:
train_loss = []
for epoch in range(20000):
    optimizer.zero_grad()
    outputs = net(torch.tensor(trainset.X).float().cuda())
    loss = criterion(outputs,torch.tensor(trainset.y).view(-1,1).float().cuda())
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item()/len(trainset))

In [66]:
print(f"""comprimento médio de código final no dataset de treino: {train_loss[-1]}
(compare com a entropia do dataset de treino).""")

comprimento médio de código final no dataset de treino: 0.003491003723144531
(compare com a entropia do dataset de treino).


### Pesos aprendidos são aproximadamente os parâmetros usados para gerar os dados

In [67]:
for param in net.parameters():
    print(param.data)

tensor([[12.0901, 11.7186, 11.9979, 12.1763, 12.1026]], device='cuda:0')
tensor([41.8704], device='cuda:0')


In [68]:
C.T, C0

(array([[0.32098451, 0.77295341, 0.82311742, 0.86189127, 0.7005142 ,
         0.80153656, 0.21184858]]),
 array([[0.2792105]]))

## Treinando Modelo No Pytorch com Stochastic Gradient Descent (um pedaço dos dados na memória da placa de vídeo de cada vez)

In [69]:
net = Perceptron(N)

In [70]:
net = net.cuda()

In [71]:
trainset = CustomDataset(Xt,yt)
validset = CustomDataset(Xc,yc)

In [72]:
criterion = Log2BCELoss(reduction="sum")
optimizer = torch.optim.SGD(net.parameters(), lr=0.00001)

In [73]:

batch_size = 100000

train_loss, valid_loss = [], []

for epoch in range(200):  # loop over the dataset multiple times

    for phase in ['train','valid']:

        if phase == 'train':
            net.train(True)
            dataloader = torch.utils.data.DataLoader(
                trainset,batch_size=batch_size,shuffle=True,num_workers=2)
        else:
            net.train(False)
            dataloader=torch.utils.data.DataLoader(
                validset,batch_size=batch_size,shuffle=False,num_workers=2)
            
        running_loss = 0.0
        for data in dataloader: #tqdm(dataloader):
            
            X_b,y_b= data
            X_b = X_b.float().cuda()
            y_b = y_b.float().cuda()
            
            if phase == 'train':
                optimizer.zero_grad()
                outputs = net(X_b.float())
                loss = criterion(outputs,y_b.view(-1,1).float())
                loss.backward()
                optimizer.step()
            else:
                with torch.no_grad():
                    outputs = net(X_b)
                    loss = criterion(outputs, y_b)

            running_loss += loss.item()

        final_loss = running_loss / len(dataloader.dataset)
        if phase=='train':
            train_loss.append(final_loss)
        else:
            valid_loss.append(final_loss)
            
        print("epoch :" , epoch, ", phase :", phase, ", loss :", final_loss)

print('Finished Training')

epoch : 0 , phase : train , loss : 0.946118046875
epoch : 0 , phase : valid , loss : 0.947739296875
epoch : 1 , phase : train , loss : 0.9273425
epoch : 1 , phase : valid , loss : 0.931678828125
epoch : 2 , phase : train , loss : 0.916495546875
epoch : 2 , phase : valid , loss : 0.92316109375
epoch : 3 , phase : train , loss : 0.90713515625
epoch : 3 , phase : valid , loss : 0.913988203125
epoch : 4 , phase : train , loss : 0.89854734375
epoch : 4 , phase : valid , loss : 0.905899296875
epoch : 5 , phase : train , loss : 0.890542890625
epoch : 5 , phase : valid , loss : 0.898238828125
epoch : 6 , phase : train , loss : 0.8830028125
epoch : 6 , phase : valid , loss : 0.891033046875
epoch : 7 , phase : train , loss : 0.875839609375
epoch : 7 , phase : valid , loss : 0.884167734375
epoch : 8 , phase : train , loss : 0.86898640625
epoch : 8 , phase : valid , loss : 0.877589609375
epoch : 9 , phase : train , loss : 0.862392265625
epoch : 9 , phase : valid , loss : 0.87124703125
epoch : 10 ,

epoch : 81 , phase : valid , loss : 0.5682123046875
epoch : 82 , phase : train , loss : 0.5578233203125
epoch : 82 , phase : valid , loss : 0.56518125
epoch : 83 , phase : train , loss : 0.5548781640625
epoch : 83 , phase : valid , loss : 0.5621755859375
epoch : 84 , phase : train , loss : 0.5519571484375
epoch : 84 , phase : valid , loss : 0.559195234375
epoch : 85 , phase : train , loss : 0.54906046875
epoch : 85 , phase : valid , loss : 0.55623953125
epoch : 86 , phase : train , loss : 0.5461872265625
epoch : 86 , phase : valid , loss : 0.5533084375
epoch : 87 , phase : train , loss : 0.54333765625
epoch : 87 , phase : valid , loss : 0.5504015625
epoch : 88 , phase : train , loss : 0.5405112890625
epoch : 88 , phase : valid , loss : 0.5475191015625
epoch : 89 , phase : train , loss : 0.537708125
epoch : 89 , phase : valid , loss : 0.5446603125
epoch : 90 , phase : train , loss : 0.5349277734375
epoch : 90 , phase : valid , loss : 0.5418255859375
epoch : 91 , phase : train , loss : 0

epoch : 161 , phase : train , loss : 0.384312421875
epoch : 161 , phase : valid , loss : 0.388752109375
epoch : 162 , phase : train , loss : 0.3827125
epoch : 162 , phase : valid , loss : 0.3871307421875
epoch : 163 , phase : train , loss : 0.3811242578125
epoch : 163 , phase : valid , loss : 0.3855209765625
epoch : 164 , phase : train , loss : 0.3795471875
epoch : 164 , phase : valid , loss : 0.3839225
epoch : 165 , phase : train , loss : 0.377981171875
epoch : 165 , phase : valid , loss : 0.38233546875
epoch : 166 , phase : train , loss : 0.376426171875
epoch : 166 , phase : valid , loss : 0.3807595703125
epoch : 167 , phase : train , loss : 0.3748821875
epoch : 167 , phase : valid , loss : 0.379195078125
epoch : 168 , phase : train , loss : 0.3733490625
epoch : 168 , phase : valid , loss : 0.377641328125
epoch : 169 , phase : train , loss : 0.3718266015625
epoch : 169 , phase : valid , loss : 0.3760986328125
epoch : 170 , phase : train , loss : 0.3703147265625
epoch : 170 , phase : 

In [74]:
print(f"""comprimento médio de código final no dataset de treino: {train_loss[-1]}
(compare com a entropia do dataset de treino).""")

print(f"""comprimento médio de código final no dataset de validação: {valid_loss[-1]}
(compare com a entropia do dataset de validação).""")

comprimento médio de código final no dataset de treino: 0.3306883984375
(compare com a entropia do dataset de treino).
comprimento médio de código final no dataset de validação: 0.334433671875
(compare com a entropia do dataset de validação).


### Pesos aprendidos são aproximadamente os parâmetros usados para gerar os dados

In [75]:
for param in net.parameters():
    print(param.data)

tensor([[2.7900, 2.2502, 2.6719, 2.8991, 2.8084]], device='cuda:0')
tensor([9.1377], device='cuda:0')


In [76]:
C.T, C0

(array([[0.32098451, 0.77295341, 0.82311742, 0.86189127, 0.7005142 ,
         0.80153656, 0.21184858]]),
 array([[0.2792105]]))